In [139]:
from dash import Dash,html,dcc
import plotly.express as px
import pandas as pd
import numpy as np
from dash.dependencies import Input,Output,State
import dash_bootstrap_components as dbc

# insta_general

In [140]:
df = pd.read_csv('outputs/insta_general.csv', header=0)

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   level_0              23 non-null     int64  
 1   index                23 non-null     int64  
 2   username             23 non-null     object 
 3   name                 23 non-null     object 
 4   category             23 non-null     object 
 5   followers            23 non-null     int64  
 6   followings           23 non-null     int64  
 7   posts                23 non-null     int64  
 8   tier                 23 non-null     object 
 9   avatar               23 non-null     object 
 10  avg_engagement_rate  23 non-null     float64
 11  total likes          23 non-null     int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 2.3+ KB


In [5]:
df0 = df.sort_values(by=['followers'])
followers_Bar = px.bar(df0 , y= 'name', x='followers', orientation='h')

In [6]:
df0 = df.sort_values(by=['posts'])
postsNum_Bar = px.bar(df0 , x= 'name', y='posts')

In [7]:
dir(px.colors.sequential)

['Aggrnyl',
 'Aggrnyl_r',
 'Agsunset',
 'Agsunset_r',
 'Blackbody',
 'Blackbody_r',
 'Bluered',
 'Bluered_r',
 'Blues',
 'Blues_r',
 'Blugrn',
 'Blugrn_r',
 'Bluyl',
 'Bluyl_r',
 'Brwnyl',
 'Brwnyl_r',
 'BuGn',
 'BuGn_r',
 'BuPu',
 'BuPu_r',
 'Burg',
 'Burg_r',
 'Burgyl',
 'Burgyl_r',
 'Cividis',
 'Cividis_r',
 'Darkmint',
 'Darkmint_r',
 'Electric',
 'Electric_r',
 'Emrld',
 'Emrld_r',
 'GnBu',
 'GnBu_r',
 'Greens',
 'Greens_r',
 'Greys',
 'Greys_r',
 'Hot',
 'Hot_r',
 'Inferno',
 'Inferno_r',
 'Jet',
 'Jet_r',
 'Magenta',
 'Magenta_r',
 'Magma',
 'Magma_r',
 'Mint',
 'Mint_r',
 'OrRd',
 'OrRd_r',
 'Oranges',
 'Oranges_r',
 'Oryel',
 'Oryel_r',
 'Peach',
 'Peach_r',
 'Pinkyl',
 'Pinkyl_r',
 'Plasma',
 'Plasma_r',
 'Plotly3',
 'Plotly3_r',
 'PuBu',
 'PuBuGn',
 'PuBuGn_r',
 'PuBu_r',
 'PuRd',
 'PuRd_r',
 'Purp',
 'Purp_r',
 'Purples',
 'Purples_r',
 'Purpor',
 'Purpor_r',
 'Rainbow',
 'Rainbow_r',
 'RdBu',
 'RdBu_r',
 'RdPu',
 'RdPu_r',
 'Redor',
 'Redor_r',
 'Reds',
 'Reds_r',
 'Sunset

In [153]:
categories_followers_pie = px.pie(df, names='category', values='followers',title = "Most followed category" ,color_discrete_sequence= px.colors.sequential.Purpor)
categories_followers_pie

In [9]:
tier_pie = px.bar(df['tier'].values, color_discrete_sequence= px.colors.sequential.gray)
tier_pie

In [10]:
df0 = df.sort_values(by=['avg_engagement_rate'])
er_Bar = px.bar(df0 , x= 'username', y='avg_engagement_rate')

# insta_detailed

In [23]:
df_detailed = pd.read_csv('outputs/insta_detailed_processed.csv', header=0)

In [24]:
df_detailed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                460 non-null    int64  
 1   username             460 non-null    object 
 2   post                 460 non-null    object 
 3   likes                460 non-null    int64  
 4   type                 460 non-null    object 
 5   captions             460 non-null    object 
 6   date                 460 non-null    object 
 7   time                 460 non-null    object 
 8   engagement_rate      460 non-null    float64
 9   avg_engagement_rate  460 non-null    float64
dtypes: float64(2), int64(2), object(6)
memory usage: 36.1+ KB


In [25]:
df_detailed['date']= pd.to_datetime(df_detailed['date'])

In [26]:
df_detailed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   index                460 non-null    int64         
 1   username             460 non-null    object        
 2   post                 460 non-null    object        
 3   likes                460 non-null    int64         
 4   type                 460 non-null    object        
 5   captions             460 non-null    object        
 6   date                 460 non-null    datetime64[ns]
 7   time                 460 non-null    object        
 8   engagement_rate      460 non-null    float64       
 9   avg_engagement_rate  460 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 36.1+ KB


In [15]:
posts_by_type_pie = px.pie(df_detailed, names='type')

In [18]:
#df_detailed['datetimes'] = pd.to_datetime(df_detailed['datetimes'])

In [19]:
'''Likes_scatter = px.scatter(df_detailed,
                           x='datetimes',y='likes',
                           color='username',
                           hover_name='username',
                           width=1000,height=1000,
                           title='Date and Time of Post VS Likes',
                           symbol='username',
                           template='seaborn')'''

"Likes_scatter = px.scatter(df_detailed,\n                           x='datetimes',y='likes',\n                           color='username',\n                           hover_name='username',\n                           width=1000,height=1000,\n                           title='Date and Time of Post VS Likes',\n                           symbol='username',\n                           template='seaborn')"

# Dash App

In [119]:
# lucky hour
alllikes = [] 
max_like = []
hours = []
lucky_hour = []
for user in df_detailed['username'].unique():
    alllikes = df_detailed[df_detailed['username'] == user]['likes'].tolist()
    max_like.append(max(alllikes))
    hours = df_detailed[df_detailed['username'] == user]['time'].tolist()
    lucky_hour.append(hours[np.argmax(alllikes)])

In [209]:
#header
card_content0 = [html.H1("Young Content Creators on Instagram", \
         style ={'font-size': '600%', 'text-align': 'center', \
                 'height': '50%', 'color': '#8a5aa0', 'font-family': 'Cursive'})]

def CustomCard(*args, **kwargs):
    option = kwargs['color']
    default_color = "red"
    default_color_light = ""
    default_color_dark = ""
    if option =='primary':
        kwargs.setdefault("style", {"background-color": default_color, "background": "linear-gradient(to top right, orange, deeppink)"})
    elif option =='secondary':
        kwargs.setdefault("style", {"background-color": default_color, "background": "linear-gradient(to bottom left, orange, deeppink)"})
    elif option =='info':
        kwargs.setdefault("style", {"background-color": default_color, "background": "linear-gradient(to top left, orange, deeppink)"})
    else:
        kwargs.setdefault("style", {"background-color": default_color, "background": "linear-gradient(to bottom right, orange, deeppink)"})
    return dbc.Card(*args, **kwargs)

def CustomButton(*args, **kwargs):

    option = kwargs['color']
    default_color = "red"
    default_color_light = ""
    default_color_dark = ""
    if option =='primary':
        kwargs.setdefault("style", {"background-color": default_color, "background": "linear-gradient(to top right, orange, deeppink)"})
    elif option =='secondary':
        kwargs.setdefault("style", {"background-color": default_color, "background": "linear-gradient(to bottom left, orange, deeppink)"})
    elif option =='info':
        kwargs.setdefault("style", {"background-color": default_color, "background": "linear-gradient(to top left, orange, deeppink)"})
    else:
        kwargs.setdefault("style", {"background-color": default_color, "background": "linear-gradient(to bottom right, orange, deeppink)"})
    return dbc.Button(*args, **kwargs)

#Big Numbers
card_content1 = [ dbc.CardHeader("Combined Followers"),
                 dbc.CardBody([html.H5(str(df['followers'].sum()), className="card-title")], style ={'text-align': 'center'}),]
card_content2 = [ dbc.CardHeader("Total Posts"),
                 dbc.CardBody([html.H5(str(df['posts'].sum()), className="card-title")], style ={'text-align': 'center'}),]
card_content3 = [ dbc.CardHeader("Total Likes"),
                 dbc.CardBody([html.H5(str(df_detailed['likes'].sum()), className="card-title")], style ={'text-align': 'center'}),]
card_content4 = [ dbc.CardHeader("Highest Engagement Rate"),
                 dbc.CardBody([html.H5(str(round(df['avg_engagement_rate'].max(),2)), className="card-title")], style ={'text-align': 'center'}),]

#photo scroll
photos = dbc.Col([
             dbc.Card([
                dbc.Row([
                    dbc.Col([
                        dbc.CardImg(src=f"/assets/insta.png", top=True,style={"width":"120px", 'clip-path': 'circle(42%)','cursor': 'pointer'}),
                        dbc.CardBody(html.P("Instagram", style = {'text-align': 'center'}))]) ,
                *[
                    dbc.Col([
                        dbc.CardImg(src=f"/assets/{i}.jpg", top=True,style={"width":"120px", 'clip-path': 'circle(42%)', 'cursor': 'pointer'}),
                        dbc.CardBody(html.P(df.username[i], style = {'text-align': 'center'}))]) for i in range(df.shape[0])]
                ], style={"flexWrap":"nowrap" }),
            ], style={'overflowX': 'scroll',  })
        ], style={"display": "flex", 'padding': '5px'})

# pie + scatter
card_content5 = [html.Div(dcc.Graph(figure=categories_followers_pie), style = {'background-color' : '#fff7f0'}) ]
                
card_content6 = [dcc.Graph(id='graph4'),
     dcc.Dropdown(options=[{'label': str(username), 'value': str(username)} \
                                for username in df_detailed['username'].unique()],id='ddown3',multi=True,)]

# Bars
card_content7 = [html.H4("Who is most followed?", style={'text-align':'center','color': '#8a5aa0'}),dcc.Graph(id='graph1')]
card_content8 = [html.H4("Who posts the most?", style={'text-align':'center','color': '#8a5aa0'}),dcc.Graph(id='graph2')]
card_content9 = [html.H4("Who has the highest Engagement Rate?", style={'text-align':'center','color': '#8a5aa0'}),
dcc.Graph(id='graph3')]

card_content10 =[ dcc.Dropdown(options=[{'label': str(category), 'value': str(category)} \
                          for category in df['category'].unique()], id='ddown1',
                ),
                 dcc.Dropdown(options=[{'label': str(tier), 'value': str(tier)} \
                                       for tier in df['tier'].unique()], id='ddown2',) ]

#Discover Posts
card_content_posts = [ html.H2("Discover Posts",style = {'text-align': 'center', 'color': '#8a5aa0'}),
                 dbc.CardBody([dbc.Card(id='card1'),
     dcc.Dropdown(options=[{'label': str(username), 'value': str(username)} \
                                for username in df['username'].unique()],id='ddown4',),
     dcc.Dropdown(id='ddown5',),]),]



In [ ]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

@app.callback(
    Output(component_id='graph1',component_property='figure'),
    Output(component_id='graph2',component_property='figure'),
    Output(component_id='graph3',component_property='figure'),
    Input(component_id='ddown1',component_property='value'),
    Input(component_id='ddown2',component_property='value'),
)
def update_My_Div(category,tier):
    if category==None and tier==None:
        df0 = df.sort_values(by=['followers'] , ascending=False)
        df1 = df.sort_values(by=['posts'], ascending=False)
        df2 = df.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'username', y='followers', color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        postsNum_Bar = px.bar(df1 , x= 'username', y='posts', color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate' , color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        return followers_Bar,postsNum_Bar, er_Bar
    elif tier==None:
        df0 = df[df['category'] == category]
        df0 = df0.sort_values(by=['followers'], ascending=False)
        df1 = df0.sort_values(by=['posts'], ascending=False)
        df2 = df0.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'username', y='followers' , color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        postsNum_Bar = px.bar(df1 , x= 'username', y='posts' , color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate', color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        return followers_Bar,postsNum_Bar, er_Bar
    elif category==None:
        df0 = df[df['tier'] == tier]
        df0 = df0.sort_values(by=['followers'], ascending=False)
        df1 = df0.sort_values(by=['posts'], ascending=False)
        df2 = df0.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'username', y='followers', color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        postsNum_Bar = px.bar(df1 , x= 'username', y='posts' , color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate', color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        return followers_Bar,postsNum_Bar,er_Bar
        
    else:
        df0 = df[np.logical_and(df['category'] == category,df['tier'] == tier)]
        df0 = df0.sort_values(by=['followers'], ascending=False)
        df1 = df0.sort_values(by=['posts'], ascending=False)
        df2 = df0.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'username', y='followers', color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        postsNum_Bar = px.bar(df1 , x= 'username', y='posts' , color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate', color = 'username', color_discrete_sequence= px.colors.sequential.Purpor)
        return followers_Bar,postsNum_Bar, er_Bar
    
@app.callback(
    Output(component_id='graph4',component_property='figure'),
    Input(component_id='ddown3',component_property='value'),
)
def update_My_Div(username):
    if username==None or len(username)==0:
        Likes_scatter = px.scatter(df_detailed,
                        x='date',y='likes',
                        color='username',
                        hover_name='username',
                        title='Date VS Likes',
                        symbol='username',
                        color_discrete_sequence= px.colors.sequential.Purpor_r)
        return Likes_scatter
    else:
        df_scatter = df_detailed[np.isin(df_detailed.username,username)]
        Likes_scatter = px.scatter(df_scatter,
                        x='date',y='likes',
                        color='username',
                        hover_name='username',
                        title='Date VS Likes',
                        symbol='username',
                        color_discrete_sequence= px.colors.sequential.Purpor_r)
        return Likes_scatter

@app.callback(
    Output(component_id='ddown5',component_property='options'),
    Input(component_id='ddown4',component_property='value'),
)

def update_My_Div(username):
    df_name = df_detailed[df_detailed['username']==username]
    options=[{'label': str(date)[:10], 'value': str(date)}  for date in df_name['date'].unique()]

    return options

@app.callback(
    Output(component_id='card1',component_property='children'),
    Input(component_id='ddown4',component_property='value'),
    Input(component_id='ddown5',component_property='value'),
)
def update_My_Div(username, date):
    
    df_name = df_detailed[df_detailed['username']==username]
    
    if username == None:
        children = [dbc.CardImg(src="https://raw.githubusercontent.com/ayahhhany/Instagram-Dashboard/main/assets/insta.png", top=True,style = {"width":"200px", 'clip-path': 'circle(42%)', 'cursor': 'pointer', 'margin-left': 'auto', 'margin-right': 'auto'})]
    elif date == None or ((df_name['date']!=date).all()) :
        children = [ dbc.CardImg(src = df[df['username'] == username].iloc[0,9], top=True, style={"width":"200px", 'clip-path': 'circle(42%)', 'cursor': 'pointer','margin-left': 'auto', 'margin-right': 'auto'}),
                    dbc.CardHeader(f"Lucky Hour is around {lucky_hour[df[df['username'] == username].index[0]][:2]} (GMT+2) "),
                    dbc.CardBody([html.H4(username, className="card-title"),]),
                    
                   ]
    else:
        children = [ dbc.CardImg(src = df[df['username'] == username].iloc[0,9], top=True, style = {"width":"200px", 'clip-path': 'circle(42%)', 'cursor': 'pointer', 'margin-left': 'auto', 'margin-right': 'auto'}),
                    dbc.CardHeader(f"Lucky Hour is around {lucky_hour[df[df['username'] == username].index[0]][:2]} (GMT+2) "),

                    dbc.CardBody([html.H4(username, className="card-title"),]),
                    html.P((df_name[df_name['date'] == date].iloc[0,5]), className="card-text",),
                    CustomButton("Open Post", color="secondary", href=df_name[df_name['date'] == date].iloc[0,2],),
                   ]

    return children

    


app.layout =html.Div(children=[
    
 html.Div(
    [
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_content0)),
            ],
            className="mb-4", 
        ),
        dbc.Row(
            [
                dbc.Col(CustomCard(card_content1, color="primary", inverse=True )),
                dbc.Col(CustomCard(card_content2, color="secondary", inverse=True)),
                dbc.Col(CustomCard(card_content3, color="info", inverse=True)),
                dbc.Col(CustomCard(card_content4, color="success", inverse=True)),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(dbc.Card(photos)),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(card_content5, width = 4),
                dbc.Col(card_content_posts , width=3),
                dbc.Col(card_content6, width =5),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(card_content7,  width=4),
                dbc.Col(card_content8,  width=4),
                dbc.Col(card_content9,  width=4),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(card_content10),
            ],
            className="mb-4",
        ),
    ]
)
    ])
app.run_server()



Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [01/May/2022 06:35:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 06:35:54] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_1_0m1651242495.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 06:35:54] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_3_0m1648105790.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 06:35:54] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_3_0m1648105790.js HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 06:35:54] "GET /_dash-component-suites/dash/html/dash_html_components.v2_0_2m1648105790.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 06:35:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 06:35:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 06:35:54] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2022 06:35:54]

127.0.0.1 - - [01/May/2022 09:13:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 09:13:03] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2022 09:13:03] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 09:13:03] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2022 09:13:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 09:13:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 09:13:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 09:13:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 09:13:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 09:13:05] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_3_0m1648105790.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 09:13:05] "GET /_dash-component-suites/da